In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import os
import bokeh.plotting as bpl
import bokeh.palettes as bpt
import json

In [3]:
bpl.output_notebook()

Loading BokehJS ...

In [4]:
zurb = pd.read_csv("./datos_SUNs/Proy_ZM_2015_mun.csv")

In [5]:
zurb["CVEMUN_RES"] = zurb["CVE_MUN"].astype(str).str.zfill(5)

Para ejecutar estos análisis primero se tiene que ejecutar el notebook del archivo "datos_historicos.ipynb", ese notebook permite descargar los archivos de datos con los que trabaja este notebook.

In [6]:
primera_fecha = "04/12/2020" #No hay datos antes del 04/12/2020 en las bases federales
ultima_fecha = "08/05/2020"

In [7]:
fechas = pd.date_range(start=primera_fecha, end=ultima_fecha)

Carga de datos históricos y actuales y conversión a DataFrame, para cada archivo se agrega una columna al dataframe que hace referencia a la fecha de la publicación del archivo, el país va a estar roto por problemas de encoding. En libre office se pueden abrir como utf8 pero python detecta un caracter que no puede decodificar y no abre los archivos. Se están forzando a 'latin1' pero entonces los acentos no son correctos.

In [8]:
fechas[-1]

Timestamp('2020-08-05 00:00:00', freq='D')

In [9]:
def proceso_data(data):
    data["FECHA_ARCHIVO"] = pd.to_datetime(data["FECHA_ARCHIVO"],format="%Y-%m-%d")
    data["FECHA_ACTUALIZACION"] = pd.to_datetime(data["FECHA_ACTUALIZACION"],format="%Y-%m-%d")
    data["FECHA_INGRESO"] = pd.to_datetime(data["FECHA_INGRESO"],format="%Y-%m-%d")
    data["FECHA_DEF"] = pd.to_datetime(data["FECHA_DEF"].replace({"9999-99-99":None}),format="%Y-%m-%d")
    data["FECHA_SINTOMAS"] = pd.to_datetime(data["FECHA_SINTOMAS"],format="%Y-%m-%d")
    data["ENTIDAD_UM"] = data["ENTIDAD_UM"].astype(str).str.zfill(2)
    data["ENTIDAD_NAC"] = data["ENTIDAD_NAC"].astype(str).str.zfill(2)
    data["ENTIDAD_RES"] = data["ENTIDAD_RES"].astype(str).str.zfill(2)
    data["CVENAC_RES"] = "00000"
    data["CVEENT_RES"] = data["ENTIDAD_RES"].astype(str)
    data["MUNICIPIO_RES"] = data["MUNICIPIO_RES"].fillna(0).astype(int).astype(str).str.zfill(3)
    data["CVEMUN_RES"] = data["ENTIDAD_RES"] + data["MUNICIPIO_RES"]
    data = pd.merge(data,zurb[["CVEMUN_RES","CVE_SUN"]],on="CVEMUN_RES",how="left")
    return data

In [31]:
def get_data(fecha):
    fechacsv = fecha.strftime("%y%m%d")+"COVID19MEXICO.csv"
    fechacad = fecha.strftime("%Y-%m-%d")
    try: 
        data = pd.read_csv('./datos_federales/' + fechacsv,encoding="latin1", low_memory=False)
        data.loc[:,"FECHA_ARCHIVO"] = dt.datetime.strptime(fechacsv[:6],"%y%m%d").date()
        return (True,proceso_data(data))
    except:
        print("Extraccion de datos para " + fechacsv + " falló")
        return (False,None)

In [11]:
def reportados(frame,colname):
    frames = []
    for columna in ["CVENAC_RES","CVEENT_RES","CVEMUN_RES","CVE_SUN"]:
        frames.append(frame.groupby([columna]).size().to_frame(colname))
    return pd.concat(frames)

In [12]:
def por_fecha(frame,colfecha):
    frames = []
    for columna in ["CVENAC_RES","CVEENT_RES","CVEMUN_RES","CVE_SUN"]:
        frames.append(frame.groupby([columna,colfecha]).size().to_frame("casos").reset_index().pivot(index=colfecha,columns=columna,values="casos"))
    concatenado = pd.concat(frames,axis=1)
    nuevo_indice = pd.date_range(concatenado.index.min(), concatenado.index.max(), freq='1D')
    return concatenado.reindex(nuevo_indice)

In [30]:
estu_frames = []
conf_frames = []
nega_frames = []
sosp_frames = []
difu_frames = []
acti_frames = []
for fecha in fechas:
    fechacad = fecha.strftime("%Y-%m-%d")
    flag,data = get_data(fecha)
    if not (flag):
        estudiados = data
        estu_frames.append(reportados(estudiados,fechacad))
        confirmados = data[(data["RESULTADO"]==1)]
        conf_frames.append(reportados(confirmados,fechacad))
        negativos = data[(data["RESULTADO"]==2)]
        nega_frames.append(reportados(negativos,fechacad))
        sospechosos = data[(data["RESULTADO"]==3)]
        sosp_frames.append(reportados(sospechosos,fechacad))
        difuntos = confirmados[confirmados["FECHA_DEF"].notna()]
        difu_frames.append(reportados(difuntos,fechacad))
        activos = confirmados[(fecha - confirmados["FECHA_SINTOMAS"]) < pd.Timedelta("14 days")]
        acti_frames.append(reportados(activos,fechacad))

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
estu_report = pd.concat(estu_frames,axis=1)
estu_report.index.name="CVEGEO"
conf_report = pd.concat(conf_frames,axis=1)
conf_report.index.name="CVEGEO"
nega_report = pd.concat(nega_frames,axis=1)
nega_report.index.name="CVEGEO"
sosp_report = pd.concat(sosp_frames,axis=1)
sosp_report.index.name="CVEGEO"
difu_report = pd.concat(difu_frames,axis=1)
difu_report.index.name="CVEGEO"
acti_report = pd.concat(acti_frames,axis=1)
acti_report.index.name="CVEGEO"

In [ ]:
estu_report.to_csv("./datos_procesados/estu_report.csv")
conf_report.to_csv("./datos_procesados/conf_report.csv")
nega_report.to_csv("./datos_procesados/nega_report.csv")
sosp_report.to_csv("./datos_procesados/sosp_report.csv")
difu_report.to_csv("./datos_procesados/difu_report.csv")
acti_report.to_csv("./datos_procesados/acti_report.csv")

In [ ]:
acti_report.loc[:,"2020-06-30"]

Estructura para sacar archivos por fecha de corte

In [129]:
conf_frames = []
difu_frames = []
for fecha in fechas:
    fechacad = fecha.strftime("%Y-%m-%d")
    data = get_data(fecha)
    confirmados = data[(data["RESULTADO"]==1)]
    conf_frames.append(reportados(confirmados,fechacad))
    sal_conf = por_fecha(confirmados,"FECHA_SINTOMAS").T
    sal_conf.index.name="CVEGEO"
    if not os.path.isfile("./datos_procesados/conf_sint_" + fecha.strftime("%d-%m") + ".csv"):
        sal_conf.to_csv("./datos_procesados/conf_sint_" + fecha.strftime("%d-%m") + ".csv")
    difuntos = confirmados[confirmados["FECHA_DEF"].notna()]
    difu_frames.append(reportados(difuntos,fechacad))
    sal_difu = por_fecha(difuntos,"FECHA_DEF").T
    sal_difu.index.name="CVEGEO"
    if not os.path.isfile("./datos_procesados/difu_sint_" + fecha.strftime("%d-%m") + ".csv"):
        sal_difu.to_csv("./datos_procesados/difu_sint_" + fecha.strftime("%d-%m") + ".csv")